In [171]:
import pandas as pd
import os
import neo4j
import numpy as np

# Read books data

In [193]:
df = pd.read_json('goodreads_books_poetry.json', lines=True)

In [194]:
df.head(2)

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[],US,eng,"[{'count': '8', 'name': 'to-read'}, {'count': ...",,false,3.83,,...,11,,1887,https://www.goodreads.com/book/show/16037549-v...,https://images.gr-assets.com/books/1348176637m...,16037549,3,5212748,Vision of Sir Launfal and Other Poems,Vision of Sir Launfal and Other Poems
1,0811223981,2,[],US,,"[{'count': '100', 'name': 'to-read'}, {'count'...",,false,3.83,B00U2WY9U8,...,4,,2015,https://www.goodreads.com/book/show/22466716-f...,https://images.gr-assets.com/books/1404958407m...,22466716,37,41905435,Fairy Tales: Dramolettes,Fairy Tales: Dramolettes


In [195]:
df.columns

Index(['isbn', 'text_reviews_count', 'series', 'country_code', 'language_code',
       'popular_shelves', 'asin', 'is_ebook', 'average_rating', 'kindle_asin',
       'similar_books', 'description', 'format', 'link', 'authors',
       'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'edition_information', 'publication_year', 'url',
       'image_url', 'book_id', 'ratings_count', 'work_id', 'title',
       'title_without_series'],
      dtype='object')

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36514 entries, 0 to 36513
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   isbn                  36514 non-null  object 
 1   text_reviews_count    36514 non-null  int64  
 2   series                36514 non-null  object 
 3   country_code          36514 non-null  object 
 4   language_code         36514 non-null  object 
 5   popular_shelves       36514 non-null  object 
 6   asin                  36514 non-null  object 
 7   is_ebook              36514 non-null  object 
 8   average_rating        36514 non-null  float64
 9   kindle_asin           36514 non-null  object 
 10  similar_books         36514 non-null  object 
 11  description           36514 non-null  object 
 12  format                36514 non-null  object 
 13  link                  36514 non-null  object 
 14  authors               36514 non-null  object 
 15  publisher          

In [197]:
df = df.replace('',np.nan)

In [198]:
df.drop(columns = ['popular_shelves','isbn','language_code','asin','kindle_asin','edition_information'], inplace = True)

In [199]:
for index, col in enumerate(df.columns):
    print(col, df[col].isna().sum()*100/df.shape[0])

text_reviews_count 0.0
series 0.0
country_code 0.0
is_ebook 0.0
average_rating 0.0
similar_books 0.0
description 20.611272388672838
format 19.23919592485074
link 0.0
authors 0.0
publisher 16.522429752971462
num_pages 20.553760201566522
publication_day 40.95963192200252
isbn13 22.33389932628581
publication_month 35.65755600591554
publication_year 15.928137152872871
url 0.0
image_url 0.0
book_id 0.0
ratings_count 0.0
work_id 0.0
title 0.005477351152982418
title_without_series 0.005477351152982418


In [200]:
df.text_reviews_count.astype('int64')
df.average_rating.astype('float64')
df.num_pages.astype('float64')
df.publication_month.astype('float64')
df.publication_year.astype('float64')

0        1887.0
1        2015.0
2        2008.0
3        1964.0
4        2015.0
          ...  
36509       NaN
36510       NaN
36511       NaN
36512    2013.0
36513    1996.0
Name: publication_year, Length: 36514, dtype: float64

In [201]:
df['format'].fillna(df['format'].value_counts().index[0], inplace = True)
df['publisher'].fillna(df['publisher'].value_counts().index[0], inplace = True)
df['num_pages'].fillna(df['num_pages'].value_counts().index[0], inplace = True)
df['publication_day'].fillna(df['publication_day'].value_counts().index[0], inplace = True)
df['isbn13'].fillna(df['isbn13'].value_counts().index[0], inplace = True)
df['publication_month'].fillna(df['publication_month'].value_counts().index[0], inplace = True)
df['publication_year'].fillna(df['publication_year'].value_counts().index[0], inplace = True)
df['title'].fillna(df['title'].value_counts().index[0], inplace = True)
df['title_without_series'].fillna(df['title_without_series'].value_counts().index[0], inplace = True)
df['description'].fillna(df['description'].value_counts().index[0], inplace = True)

In [202]:
df.publisher.value_counts().head(25)

W. W. Norton  Company                          6353
Farrar, Straus and Giroux                       305
Penguin Classics                                302
Penguin Books                                   291
New Directions                                  261
Copper Canyon Press                             240
Oxford University Press, USA                    240
Knopf                                           227
Graywolf Press                                  184
W. W. Norton & Company                          180
Createspace Independent Publishing Platform     172
Dover Publications                              168
Faber  Faber                                    165
University of Pittsburgh Press                  158
Mariner Books                                   146
Oxford University Press                         146
Faber & Faber                                   146
Ecco                                            143
University of California Press                  134
HMH Books fo

In [267]:
pubList = ['Penguin Books','New Directions',
          'Oxford University Press, USA','Graywolf Press',
          'Dover Publications', 'Faber  Faber', 'University of Pittsburgh Press', 'Mariner Books','Oxford University Press','Oxford University Press',
          'University of California Press', 'HarperCollins','Wesleyan']

In [268]:
df[df['publisher'].isin(pubList)].shape

(2120, 23)

In [269]:
df = df[df['publisher'].isin(pubList)]

## Extract authors in a dataframe

In [270]:
def extract_author(df, target_col):
    temp_df = pd.DataFrame(columns=['book_id', target_col])
    for index, row in df[['book_id', 'authors']].dropna().iterrows():
        list_of_values = row[1][0]['author_id']
        book_id = row[0]
        #print([list_of_values])
        temp_df = pd.concat([temp_df, pd.DataFrame({'book_id': book_id, target_col: [list_of_values]})])
    return temp_df

In [271]:
authors_df = extract_author(df, 'author')

In [272]:
authors_df.head(2)

,book_id,author
0,22466716,16073
0,926667,18540


In [273]:
authors_df.to_csv('authors.csv', index = False)

## Extract Publisher

In [274]:
publisher_df = df[['book_id','publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'publication_year']]

In [381]:
publisher_df.head(8)

,book_id,publisher,num_pages,publication_day,isbn13,publication_month,publication_year
1,22466716,New Directions,128,20,9780811223980,4,2015
3,926667,Mariner Books,190,18,9780156182898,3,1964
39,773136,University of Pittsburgh Press,104,31,9780822953685,3,1985
51,752936,Dover Publications,64,1,9780486264684,7,1990
59,7517330,Graywolf Press,167,20,9781555975623,7,2010
66,55427,Faber Faber,96,23,9780571174256,8,1999
75,849845,University of California Press,96,22,9780520229259,5,2001
112,7113632,Oxford University Press,262,1,9780195547726,4,1988


In [276]:
publisher_df = publisher_df[publisher_df.publisher.notna()]

In [277]:
publisher_df.to_csv('publisher.csv', index = False)

## Extract similar books

In [213]:
def extract_similarBooks(df, target_col):
    temp_df = pd.DataFrame(columns=['book_id', target_col])
    for index, row in df[['book_id', 'similar_books']].dropna().iterrows():
        list_of_values = row[1]
        book_id = row[0]
        temp_df = pd.concat([temp_df, pd.DataFrame({'book_id': book_id, target_col: list_of_values})])
    return temp_df

In [214]:
similarBooks_df = extract_similarBooks(df,'similar_books')

In [215]:
similarBooks_df.head(2)

,book_id,similar_books
0,926667,1230072
1,926667,315167


In [216]:
similarBooks_df.to_csv('similarBooks.csv', index = False)

## Extract series

In [217]:
def extract_series(df, target_col):
    temp_df = pd.DataFrame(columns=['book_id', target_col])
    for index, row in df[['book_id', 'series']].dropna().iterrows():
        list_of_values = row[1]
        book_id = row[0]
        temp_df = pd.concat([temp_df, pd.DataFrame({'book_id': book_id, target_col: list_of_values})])
    return temp_df

In [ ]:
series_df = extract_series(df, 'series')

In [ ]:
series_df.head(2)

In [ ]:
series_df.to_csv('bookSeries.csv', index = False)

## Extract books alone

In [278]:
df.columns

Index(['text_reviews_count', 'series', 'country_code', 'is_ebook',
       'average_rating', 'similar_books', 'description', 'format', 'link',
       'authors', 'publisher', 'num_pages', 'publication_day', 'isbn13',
       'publication_month', 'publication_year', 'url', 'image_url', 'book_id',
       'ratings_count', 'work_id', 'title', 'title_without_series'],
      dtype='object')

In [279]:
books_df = df[['book_id', 'text_reviews_count', 'country_code',
       'is_ebook', 'average_rating', 'description', 'format', 'link', 'url',
       'image_url', 'ratings_count', 'work_id', 'title',
       'title_without_series']]

In [374]:
books_df.head(10)

,book_id,text_reviews_count,country_code,is_ebook,average_rating,description,format,link,url,image_url,ratings_count,work_id,title,title_without_series
1,22466716,2,US,false,3.83,Fairy Tales gathers the unconventional verse d...,Paperback,https://www.goodreads.com/book/show/22466716-f...,https://www.goodreads.com/book/show/22466716-f...,https://images.gr-assets.com/books/1404958407m...,37,41905435,Fairy Tales: Dramolettes,Fairy Tales: Dramolettes
3,926667,12,US,false,3.71,A modern verse play about the search for meani...,Paperback,https://www.goodreads.com/book/show/926667.The...,https://www.goodreads.com/book/show/926667.The...,https://images.gr-assets.com/books/1382939971m...,115,995066,The Cocktail Party,The Cocktail Party
39,773136,3,US,false,4.51,Since the appearance of his first book in 1972...,Paperback,https://www.goodreads.com/book/show/773136.Win...,https://www.goodreads.com/book/show/773136.Win...,https://images.gr-assets.com/books/1328807845m...,58,759187,Winter Stars,Winter Stars
51,752936,86,US,false,4.04,Few volumes of poetry in the English language ...,Paperback,https://www.goodreads.com/book/show/752936.A_S...,https://www.goodreads.com/book/show/752936.A_S...,https://images.gr-assets.com/books/1414599637m...,1438,1548437,A Shropshire Lad,A Shropshire Lad
59,7517330,56,US,false,4.15,"In The Art of Recklessness, Dean Young's spraw...",Paperback,https://www.goodreads.com/book/show/7517330-th...,https://www.goodreads.com/book/show/7517330-th...,https://images.gr-assets.com/books/1316637843m...,468,9729336,The Art of Recklessness: Poetry as Assertive F...,The Art of Recklessness: Poetry as Assertive F...
66,55427,6,US,false,4.02,"This labyrinthine and extraordinary book, firs...",Paperback,https://www.goodreads.com/book/show/55427.The_...,https://www.goodreads.com/book/show/55427.The_...,https://s.gr-assets.com/assets/nophoto/book/11...,60,219413,The White Goddess,The White Goddess
75,849845,3,US,false,4.00,"These poems, selected from Das Buch der Bilder...",Paperback,https://www.goodreads.com/book/show/849845.Rilke,https://www.goodreads.com/book/show/849845.Rilke,https://s.gr-assets.com/assets/nophoto/book/11...,18,54762659,Rilke: Selected Poems,Rilke: Selected Poems
112,7113632,1,US,false,3.00,A collection of carefully selected poems by Au...,Paperback,https://www.goodreads.com/book/show/7113632-tw...,https://www.goodreads.com/book/show/7113632-tw...,https://images.gr-assets.com/books/1344263105m...,2,7373456,Two Centuries of Australian Poetry,Two Centuries of Australian Poetry
122,1222347,2,US,false,4.57,"One of the greatest plays of all time, the com...",Paperback,https://www.goodreads.com/book/show/1222347.Wa...,https://www.goodreads.com/book/show/1222347.Wa...,https://s.gr-assets.com/assets/nophoto/book/11...,4,2645891,Walking Distance,Walking Distance
139,704917,5,US,false,3.78,Yet in this selection of sixty-two poems chose...,Paperback,https://www.goodreads.com/book/show/704917.The...,https://www.goodreads.com/book/show/704917.The...,https://s.gr-assets.com/assets/nophoto/book/11...,36,691202,The Life Around Us: Selected Poems on Nature,The Life Around Us: Selected Poems on Nature


In [375]:
books_df.format.value_counts()

Paperback                1730
Hardcover                 325
ebook                      43
Unknown Binding             7
Mass Market Paperback       5
paper                       4
Kindle Edition              3
Audio                       1
Audio CD                    1
Hardback                    1
Name: format, dtype: int64

In [281]:
ebooks_df = books_df[books_df.is_ebook == 'true']

In [282]:
not_ebooks_df = books_df[books_df.is_ebook == 'false']

In [283]:
ebooks_df.to_csv('ebooks.csv', index = False)
#not_ebooks_df.to_csv('not_ebooks.csv', index = False)

In [284]:
not_ebooks = not_ebooks_df.copy()

In [285]:
not_ebooks.description = not_ebooks.description.apply(lambda x: x.replace('"', "'"))
not_ebooks.title = not_ebooks.title.apply(lambda x: x.replace('"', "'"))
not_ebooks.title_without_series = not_ebooks.title_without_series.apply(lambda x: x.replace('"', "'"))

In [286]:
not_ebooks.to_csv('not_ebooks.csv', index = False)

In [ ]:
not_ebooks[not_ebooks.apply(lambda row: row.astype(str).str.contains("\"").any(), axis=1)]

# Read reviews data

In [307]:
df_reviews = pd.read_json('goodreads_reviews_poetry.json', lines=True)

In [308]:
df_reviews.columns

Index(['user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')

In [ ]:
df_reviews[df_reviews.apply(lambda row: row.astype(str).str.contains("\"").any(), axis=1)]

In [311]:
df_reviews.review_text = df_reviews.review_text.apply(lambda x: x.replace('"', "'"))
# df_reviews.review_text = df_reviews.review_text.apply(lambda x: x.replace('"', "'"))

In [312]:
df_reviews.isnull().any()

user_id         False
book_id         False
review_id       False
rating          False
review_text     False
date_added      False
date_updated    False
read_at         False
started_at      False
n_votes         False
n_comments      False
dtype: bool

In [340]:
from datetime import datetime

df_reviews['date_added'] = pd.to_datetime(df_reviews['date_added'], utc=True)

In [347]:
df_reviews['date_added'] = df_reviews['date_added'].dt.date

In [348]:
df_reviews = df_reviews[['user_id', 'book_id', 'review_id', 'rating','date_added']]

In [349]:
booklist = df.book_id.unique().tolist()

In [350]:
df_reviews = df_reviews[df_reviews['book_id'].isin(booklist)]

# Users data exploration

In [379]:
reviewers = df_reviews.user_id.unique().tolist()
len(reviewers)

6206

In [380]:
df_reviews.head(10)

,user_id,book_id,review_id,rating,date_added,year
1,0ef32090550901ead25cb0ea21c4d36b,92270,2db1180992e2b0b1631a3ac5644bde84,5,2014-04-15,2014
4,af157d0205b8a901dee6d4a2aed7e6ad,70885,8dca128b8e869048a7442c18659dbece,5,2015-06-19,2015
29,7e4945a6266566e39e3f251fff0fdde1,128436,5e9edafcc95cc2526fe0830ca046d009,5,2012-06-26,2012
32,d4b1dcb35db677f20ee45225a5e43be2,17648,db5e9df7999776cab56860ed73248c12,4,2007-08-05,2007
45,34216df2a5f3846b17e3f96bea6c2ad7,20613761,6408237e977c0dbe0297ba7d0d259a09,3,2016-11-08,2016
52,c6237533dc14102048c38661051da41c,47734,42a50e67f63371ad6f8c1af316904ab8,5,2008-01-09,2008
65,5f550cc8d90df1759eda1496621187d8,20613761,2cc330737e7b15c6ca5da082f620ffac,3,2016-11-04,2016
68,5f550cc8d90df1759eda1496621187d8,143505,a930e66d3313d503d074e00cbc33b25a,2,2014-02-27,2014
88,a4f283aafe153b1997dc6a9fcd2e8e58,160192,41bd27def417e9f5493da0c5eaf8c334,4,2009-03-01,2009
92,753746d5d26fea030b33299b34ea1299,7055794,99909e9461f4889ab638d88973a462c0,2,2010-05-18,2010


In [357]:
df_reviews.to_csv('reviews1.csv', index = False)

In [368]:
df_reviews['year'] = pd.to_datetime(df_reviews['date_added'])

In [369]:
df_reviews['year'] = df_reviews['year'].dt.year

In [373]:
df_reviews['year'].value_counts()

2016    1458
2015    1352
2013    1173
2014    1147
2017    1131
2012    1032
2011     841
2008     797
2009     604
2010     582
2007     493
Name: year, dtype: int64

In [299]:
print("number of users - ",len(set(df_reviews.user_id)))
print("number of books - ", len(set(df.book_id)))
print("number of authors - ", len(set(authors_df.author)))
print("number of publishers - ", len(set(df.publisher)))
#print("number of series - ", len(set(series_df.series)))
print("number of reviews - ",len(set(df_reviews.review_id)))

number of users -  6206
number of books -  2120
number of authors -  904
number of publishers -  12
number of reviews -  10610
